In [1]:
#all imports here

from nilearn import plotting, image
import nibabel as nib
import os
import numpy as np
from pathlib import Path
import tarfile
import altair as alt
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
def sliding_windows(data, num_windows, create_figure=False):
    '''
    data: is the data that we want to be split into whindows, one subject data
    num_windows: is the number of windows we want to split the data into
    create_figure: if True, it will create a figure with the windows, if False, it will not create a figure
    '''
    len_window = 4800//num_windows
    window_matrices = []
    if create_figure:
        fig, axs = plt.subplots(num_windows//4, 4, figsize=(15, 10))
    for i in range(num_windows):
        row = i//4
        col = i % 4
        i = int(i*len_window//2)
        lil_data = data[i: i + len_window]
        sub_corr_matrix = np.corrcoef(lil_data.T)
        window_matrices.append(sub_corr_matrix) 
        if create_figure:
            img = axs[row][col].imshow(sub_corr_matrix, cmap="RdBu_r",vmin=-1, vmax=1, interpolation="none")
        # axs[row][col].title(f"corr graph in window {i} to {i+len_5_min}")
        #plt.show()
    if create_figure:
        fig.colorbar(img, ax=axs, orientation='vertical', fraction=0.02, pad=0.04, shrink = 0.4)
    return window_matrices

In [ ]:
#lets check the 99.9 percentile
def percentile(cov_matrix, percentile=99.9):
    ''' 
    find percentile

    Takes in:
    cov_matrix
    percentile = 99.9
    '''
    cov_matrix = np.array(cov_matrix)
    iu = np.triu_indices(100, 1) #get the upper triangle and and offset it to not include the diagonal
    unique_cov = abs(cov_matrix[iu])
    threshold = np.percentile(unique_cov, percentile) #threshold for 99.9 percentile
    indices = np.where((cov_matrix >= threshold) & (cov_matrix <0.999))
    pairs = np.dstack(indices)
    unique_pairs = np.array(list({tuple(np.sort(row)) for row in pairs[0]}))
    strong_correlations = {(x, y):cov_matrix[x, y] for x, y in unique_pairs}
    return strong_correlations

In [ ]:
def get_data_over_time(matrices, x, y):
    data = np.array(matrices)[:, x, y]
    data = pd.DataFrame({
        'x': range(1, len(data) + 1),  # Generate x-axis values (e.g., 1, 2, 3, ...)
        'y': data,
        'group':[f'({x},{y})']*len(data) # Use the list for y-axis values
    })
    return data
    
def plot_correlations_over_time(data):
    chart = alt.Chart(data).mark_point().encode(
    x=alt.X('x', title='Time Window'),
    y=alt.Y('y', title = 'Correlation'),
    color='group',
    tooltip=['x', 'y'],
    ).properties(title='Correlation over 12 windows', )
    return chart

In [ ]:
atlas = image.load_img("HCP_PTN1200/groupICA/groupICA_3T_HCP1200_MSMAll_d100.ica/melodic_IC_sum.nii.gz")
atlas = image.threshold_img(atlas, "99.5%") 

atlas_coords = plotting.find_probabilistic_atlas_cut_coords(atlas)

In [ ]:
def distance_between_coords(id1, id2):
    return np.sqrt(np.sum([(atlas_coords[id2][i] - atlas_coords[id1][i])**2 for i in range(3)]))

In [ ]:
# num_windows = 12
# data = X
# windows12 = sliding_windows(data, num_windows, True)